<a href="https://colab.research.google.com/github/OnyxianSoul/ElementalWeightCalculator/blob/main/Elemental_Weight_Calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><font color="#04AFDA">Elemental weight calculator</font></h1>
**A simple jupyter notebook that uses python to calculate the amount of each element in x amount of a molecule.**

It will take a few seconds the first time because its setting up the enviroment.

A warning will pop up reminding you that this notebook was not made by google, but by me. *Its a standard warning, feel free to inspect the code, it only performs the steps necessairy to produce the table.*

Credits to Bowserinator for [his periodic table in csv format](https://github.com/Bowserinator/Periodic-Table-JSON/blob/master/PeriodicTableCSV.csv) 😀

In [ ]:
# @title ## $\color{#04AFDA}{\text{Fill the molecule details and press the play button!}}$
Formula = "H2O(CH2)4Ca" # @param {type:"string"}
Amount = 100 # @param {type:"integer"}
Unit = "gr" # @param {type:"string"}

import urllib.request
import pandas as pd
import re

link = 'https://github.com/Bowserinator/Periodic-Table-JSON/blob/master/PeriodicTableCSV.csv'
dowload_link = link.replace("github.com", "raw.githubusercontent.com").replace("/blob/", "/")
file_name = "PeriodicTable.csv"
urllib.request.urlretrieve(dowload_link, file_name)
elements_df = pd.read_csv(file_name, usecols=['name', 'atomic_mass', 'symbol'])
symbols_list = elements_df['symbol'].to_list()
pattern = re.compile(r'([A-Z][a-z]*)(\d*)|(\((.+?)\)(\d*))')

def parse_formula(formula):
    counts = {symbol: 0 for symbol in symbols_list}

    def handle_match(match):
        if match[0]:  # Element
            element = match[0]
            count = int(match[1]) if match[1] else 1
            counts[element] += count
        else:  # Group
            group_formula = match[3]
            multiplier = int(match[4]) if match[4] else 1
            group_counts = parse_formula(group_formula)
            for element, count in group_counts.items():
                counts[element] += count * multiplier

    for match in pattern.findall(formula):
        handle_match(match)

    return counts


def calculate_element_mass(formula, molecule_amount, molecule_amount_unit):
    counts = parse_formula(formula)

    # Calculate the molecular weight of each element, the total mass of each element, and the proportion of the total mass of each element to the total mass of the formula
    total_mass = 0
    element_masses = {}
    element_atomic_masses = {}
    for element, count in counts.items():
        if count > 0:
            atomic_mass = elements_df.loc[elements_df['symbol'] == element, 'atomic_mass'].values[0]
            element_atomic_masses[element] = atomic_mass
            element_mass = atomic_mass * count
            element_masses[element] = element_mass
            total_mass += element_mass

    # Create a DataFrame to store the results
    results = []
    three_decimals = "{:.3f}"
    for element, element_mass in element_masses.items():
        count = counts[element]
        atomic_mass = element_atomic_masses[element]
        proportion = element_mass / total_mass
        total_in_amount = proportion * molecule_amount
        results.append([element,
                        count,
                        three_decimals.format(atomic_mass),
                        three_decimals.format(element_mass),
                        f'{three_decimals.format(element_mass)} / {three_decimals.format(total_mass)}',
                        three_decimals.format(proportion),
                        three_decimals.format(total_in_amount)])

    results_df = pd.DataFrame(results,
                              columns=['Element', 'Atoms', 'Atomic mass', 'Mass per Molecule', 'Proportion Formula', 'Proportion', f'{molecule_amount_unit} in {molecule_amount}{molecule_amount_unit} {formula}'])
    return results_df

df = calculate_element_mass(Formula, Amount, Unit).sort_values(by='Element')

from IPython.display import HTML
display(HTML(f'<h2>Molecule: <span style="color:#04AFDA">{Formula}</span></h2>'))
display(df.style.hide(axis='index').set_properties(**{'text-align': 'center'}))



Element,Atoms,Atomic mass,Mass per Molecule,Proportion Formula,Proportion,gr in 100gr H2O(CH2)4Ca
C,4,12.011,48.044,48.044 / 114.201,0.421,42.070
Ca,1,40.078,40.078,40.078 / 114.201,0.351,35.094
H,10,1.008,10.080,10.080 / 114.201,0.088,8.827
O,1,15.999,15.999,15.999 / 114.201,0.140,14.009
